In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta

import wbdata

In [2]:
def f_unique_len_sum(df_payments, metrix):
# def f_unique_len_sum(df_payments, test_groups, metrix):
    test_groups = ['TG','CG1','CG2']
    out = []
    for test_group in test_groups:
        out.append(len(set(df_payments[(df_payments['test group'] == test_group) & (df_payments[metrix] == True)]['user ID'])))
    for test_group in test_groups:
        out.append(len(df_payments[(df_payments['test group'] == test_group) & (df_payments[metrix] == True)]['user ID']))
    for test_group in test_groups:
        out.append(df_payments[(df_payments['test group'] == test_group) & (df_payments[metrix] == True)]['revenue in cents'].sum())
    idx = ['payers__TG','payers__CG1','payers__CG2','payments__TG','payments__CG1','payments__CG2','revenue__TG','revenue__CG1','revenue__CG2']
    return(pd.DataFrame(out, columns = [metrix], index = idx))

In [3]:
# https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'*')),uiState:(),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'2',params:(filters:!(('$$hashKey':'object:3287',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_FullPremListWithoutWeek_Android_run1__TG'))),label:''),('$$hashKey':'object:3296',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_FullPremListWithoutWeek_Android_run1__CG1')))),('$$hashKey':'object:3301',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_FullPremListWithoutWeek_Android_run1__CG2')))))),schema:segment,type:filters),(id:'3',params:(field:country,order:desc,orderBy:'1',size:200),schema:group,type:terms),(id:'4',params:(filters:!(('$$hashKey':'object:3189',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:0'))),label:''),('$$hashKey':'object:3198',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:1'))))),row:!t),schema:split,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))
df_installs = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1_inst.csv')

# df_payments_TG = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__TG.csv')
# df_payments_CG1 = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG1.csv')
# df_payments_CG2 = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG2.csv')

df_payments_TG_notref = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__TG_notref.csv')
df_payments_CG1_notref = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG1_notref.csv')
df_payments_CG2_notref = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG2_notref.csv')

df_payments_TG_ref = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__TG_ref.csv')
df_payments_CG1_ref = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG1_ref.csv')
df_payments_CG2_ref = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG2_ref.csv')

In [4]:
df_payments_TG_notref['ref'] = 'notref'
df_payments_CG1_notref['ref'] = 'notref'
df_payments_CG2_notref['ref'] = 'notref'

df_payments_TG_ref['ref'] = 'ref'
df_payments_CG1_ref['ref'] = 'ref'
df_payments_CG2_ref['ref'] = 'ref'

df_payments_TG = pd.concat([df_payments_TG_notref, df_payments_TG_ref])
df_payments_CG1 = pd.concat([df_payments_CG1_notref, df_payments_CG1_ref])
df_payments_CG2 = pd.concat([df_payments_CG2_notref, df_payments_CG2_ref])

In [5]:
df_installs.columns = ['test group','country','ref','users']
df_installs.loc[df_installs['test group'].map(lambda x: bool(re.search('TG',x)) ), 'test group'] = 'TG'
df_installs.loc[df_installs['test group'].map(lambda x: bool(re.search('CG1',x)) ), 'test group'] = 'CG1'
df_installs.loc[df_installs['test group'].map(lambda x: bool(re.search('CG2',x)) ), 'test group'] = 'CG2'
df_installs.loc[df_installs['ref'] == 'referral:0', 'ref'] = 'notref'
df_installs.loc[df_installs['ref'] == 'referral:1', 'ref'] = 'ref'

In [6]:
df_payments_TG['test group'] = 'TG'
df_payments_CG1['test group'] = 'CG1'
df_payments_CG2['test group'] = 'CG2'
df_payments = pd.concat([df_payments_TG,df_payments_CG1,df_payments_CG2])

In [7]:
df_payments['Product Id'] = df_payments['Product Id'].fillna('null')
df_payments['is_prem'] = df_payments['Product Id'].map(lambda x: bool(bool(re.search('prem',x)) + bool(re.search('vip',x)) + bool(re.search('trial',x))))
df_payments['is_trial'] = df_payments['Product Id'].map(lambda x:  bool(re.search('trial',x)) )
df_payments['is_coin'] = df_payments['Product Id'].map(lambda x:  bool(re.search('coin',x)) )
df_payments['is_like'] = df_payments['Product Id'].map(lambda x:  bool(re.search('like',x)) )
df_payments['is_bomb'] = df_payments['Product Id'].map(lambda x:  bool(re.search('bomb',x)) )

In [8]:
df_payments.head()

,ref ID,partner ID,ref date,user ID,registration date,payment ID,payment date,payment amount,revenue in cents,payment service ID,...,country,last app,Product Id,ref,test group,is_prem,is_trial,is_coin,is_like,is_bomb
0,NaN,NaN,NaN,117703943,2018-04-06 02:43:23,1,2018-04-20 03:42:06,393,275,147,...,UA,topfacemobappandroid,com.topface.topface.sub.vip.month.10,notref,TG,True,False,False,False,False
1,NaN,NaN,NaN,117702090,2018-04-05 23:53:22,1,2018-04-06 21:15:04,561,392,147,...,TR,topfacemobappandroid,com.topface.topface.sub.vip.month.10,notref,TG,True,False,False,False,False
2,NaN,NaN,NaN,117681392,2018-04-05 00:09:01,4,2018-04-09 02:10:45,1965,1375,147,...,KR,topfacemobappandroid,com.topface.topface.sub.trial.vip.month.13.3da...,notref,TG,True,True,False,False,False
3,NaN,NaN,NaN,117572875,2018-03-31 20:48:48,1,2018-03-31 20:49:42,2542,1779,147,...,CL,topfacemobappandroid,com.topface.topface.sub.vip.month.3.2699.v1,notref,TG,True,False,False,False,False
4,NaN,NaN,NaN,117694331,2018-04-05 16:32:24,4,2018-04-06 21:47:43,273,191,146,...,TR,topfacemobappandroid,com.topface.topface.sub.coins.month.40.5,notref,TG,False,False,True,False,False


In [9]:
# df_payments = df_payments[df_payments['ref ID'].isna()]
# df_payments = df_payments[~df_payments['ref ID'].isna()]

# df_payments = df_payments[df_payments['registration date'] < '2018-03-24']

In [10]:
df_payments['all'] = True
metrices = ['all','is_prem','is_trial','is_coin','is_like','is_bomb']

In [11]:
spend_by_types = pd.concat(list(map(lambda x: f_unique_len_sum(df_payments, x), metrices)), axis = 1)

In [12]:
spend_by_types

,all,is_prem,is_trial,is_coin,is_like,is_bomb
payers__TG,271,237,124,71,41,4
payers__CG1,349,310,135,95,53,4
payers__CG2,323,288,121,81,46,3
payments__TG,512,293,155,164,50,5
payments__CG1,662,387,174,194,66,15
payments__CG2,599,358,155,151,66,15
revenue__TG,344406,287488,151616,45586,10612,720
revenue__CG1,353503,279561,155514,59528,12579,1835
revenue__CG2,316899,248355,138913,50632,13542,2929


In [13]:
# pd.concat(list(map(lambda x: f_unique_len_sum(df_payments[~(df_payments['ref ID'] > 0)], x), metrices)), axis = 1)

In [14]:
# pd.concat(list(map(lambda x: f_unique_len_sum(df_payments[df_payments['ref ID'] > 0], x), metrices)), axis = 1)

In [15]:
# df_installs = df_installs[df_installs['ref'] == 'notref']

In [16]:
m_installs_all = df_installs.pivot_table(index='country', columns='test group', values='users', aggfunc=sum).fillna(0).applymap(float)
m_payments_all = df_payments.pivot_table(index='country', columns='test group', values='revenue in cents', aggfunc=sum).fillna(0).applymap(float)
m_payers_all = df_payments.pivot_table(index='country', columns='test group', values='user ID', aggfunc=lambda x: len(set(x))).fillna(0).applymap(float)

m_installs_notref = df_installs[df_installs['ref'] == 'notref'].pivot_table(index='country', columns='test group', values='users', aggfunc=sum).fillna(0).applymap(float)
m_payments_notref = df_payments[df_payments['ref'] == 'notref'].pivot_table(index='country', columns='test group', values='revenue in cents', aggfunc=sum).fillna(0).applymap(float)
m_payers_notref = df_payments[df_payments['ref'] == 'notref'].pivot_table(index='country', columns='test group', values='user ID', aggfunc=lambda x: len(set(x))).fillna(0).applymap(float)

m_installs_ref = df_installs[df_installs['ref'] == 'ref'].pivot_table(index='country', columns='test group', values='users', aggfunc=sum).fillna(0).applymap(float)
m_payments_ref = df_payments[df_payments['ref'] == 'ref'].pivot_table(index='country', columns='test group', values='revenue in cents', aggfunc=sum).fillna(0).applymap(float)
m_payers_ref = df_payments[df_payments['ref'] == 'ref'].pivot_table(index='country', columns='test group', values='user ID', aggfunc=lambda x: len(set(x))).fillna(0).applymap(float)


In [17]:
out = pd.concat([m_installs_all, m_payments_all, m_payers_all, m_installs_notref, m_payments_notref, m_payers_notref, m_installs_ref, m_payments_ref, m_payers_ref], axis = 1).fillna(0)
writer = pd.ExcelWriter('installs_revenue_payers.xlsx')
out.to_excel(writer,'installs_revenue_payers')
writer.save()

In [62]:
indicators = {'NY.GNP.PCAP.CD':'GNI per Capita'}

test_countries = list(m_installs_all[m_installs_all.sum(axis = 1) > 500].index)

df = wbdata.get_dataframe(indicators, country=test_countries, convert_date=False)
df.reset_index(inplace=True)
df = df[df['date'] == '2016']

df.reset_index(inplace=True)
df['country_short'] = pd.Series(test_countries)

df = df[['country_short', 'GNI per Capita']]
df.set_index('country_short', inplace=True)

In [63]:
out.merge(df, left_index=True, right_index=True)

,CG1,CG2,TG,CG1,CG2,TG,CG1,CG2,TG,CG1,...,CG1,CG2,TG,CG1,CG2,TG,CG1,CG2,TG,GNI per Capita
AE,271.0,242.0,238.0,1373.0,1474.0,1988.0,2.0,3.0,2.0,116.0,...,155.0,137.0,121.0,1373.0,1474.0,1988.0,2.0,3.0,2.0,40480.0
AR,429.0,459.0,454.0,2240.0,5597.0,3058.0,1.0,3.0,2.0,135.0,...,294.0,327.0,325.0,0.0,3357.0,1938.0,0.0,2.0,1.0,11970.0
AZ,198.0,189.0,166.0,5144.0,1381.0,2230.0,5.0,2.0,1.0,81.0,...,117.0,109.0,93.0,484.0,1381.0,2230.0,1.0,2.0,1.0,4760.0
BD,458.0,467.0,446.0,0.0,349.0,0.0,0.0,1.0,0.0,142.0,...,316.0,330.0,323.0,0.0,349.0,0.0,0.0,1.0,0.0,1330.0
BG,197.0,204.0,199.0,2903.0,4028.0,0.0,4.0,2.0,0.0,38.0,...,159.0,176.0,163.0,2903.0,3391.0,0.0,4.0,1.0,0.0,7580.0
BO,166.0,188.0,181.0,775.0,1745.0,1018.0,1.0,2.0,1.0,49.0,...,117.0,137.0,122.0,775.0,1745.0,1018.0,1.0,2.0,1.0,5600.0
BR,2595.0,2676.0,2560.0,13302.0,3735.0,6677.0,13.0,4.0,7.0,343.0,...,2252.0,2314.0,2211.0,13302.0,3735.0,5130.0,13.0,4.0,5.0,3070.0
BY,296.0,287.0,318.0,3729.0,2745.0,3208.0,4.0,4.0,3.0,194.0,...,102.0,103.0,125.0,1523.0,968.0,2572.0,2.0,1.0,2.0,8840.0
CL,409.0,380.0,421.0,3648.0,0.0,1779.0,3.0,0.0,1.0,88.0,...,321.0,274.0,311.0,3648.0,0.0,0.0,3.0,0.0,0.0,13540.0
CO,1560.0,1539.0,1549.0,2653.0,9081.0,7132.0,4.0,10.0,7.0,446.0,...,1114.0,1097.0,1128.0,2653.0,4556.0,4638.0,4.0,5.0,5.0,6310.0


In [65]:
writer = pd.ExcelWriter('installs_revenue_payers.xlsx')
out.merge(df, left_index=True, right_index=True).to_excel(writer,'installs_revenue_payers')
writer.save()

In [70]:
df_payments[df_payments['payment provider'] == 'Деньги от Google Play'].groupby(['ref','test group'])['revenue in cents'].sum()

ref     test group
notref  CG1            48583
        CG2            46846
        TG             51138
ref     CG1           181893
        CG2           141646
        TG            153460
Name: revenue in cents, dtype: int64

In [71]:
df_payments[df_payments['payment provider'] == 'Деньги от Payment Ninja'].groupby(['ref','test group'])['revenue in cents'].sum()

ref     test group
notref  CG1            29839
        CG2            29753
        TG              8190
ref     CG1            81336
        CG2            84408
        TG            123255
Name: revenue in cents, dtype: int64

In [72]:
df_payments[df_payments['payment provider'] == 'Деньги от Google Play'].groupby(['ref','test group'])['user ID'].agg(lambda x: len(set(x)))

ref     test group
notref  CG1            44
        CG2            42
        TG             51
ref     CG1           198
        CG2           161
        TG            149
Name: user ID, dtype: int64

In [73]:
df_payments[df_payments['payment provider'] == 'Деньги от Payment Ninja'].groupby(['ref','test group'])['user ID'].agg(lambda x: len(set(x)))

ref     test group
notref  CG1           18
        CG2           17
        TG             6
ref     CG1           69
        CG2           76
        TG            44
Name: user ID, dtype: int64